In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [48]:
# Test set: Average loss: 0.0186, Accuracy: 9943/10000 (99.43%) with drop, batch, and extra linear
class MNISTSmallConv(nn.Module):
    def __init__(self):
      super(MNISTSmallConv, self).__init__()
      # n = 13
      self.conv1 = nn.Conv2d(1, 16, 3, stride=2) #input -? OUtput? RF
      # n = 11
      self.conv2 = nn.Conv2d(16, 32, 3)
      # n = 5
      self.pool1 = nn.MaxPool2d(2, 2)
      # n = 3
      self.conv3 = nn.Conv2d(32, 32, 3)
      self.linear1 = nn.Linear(288, 16)
      self.linear2 = nn.Linear(16, 10)

      self.drop1 = nn.Dropout(0.1)
      self.drop2 = nn.Dropout(0.1)
      self.drop3 = nn.Dropout(0.1)
      self.drop4 = nn.Dropout(0.1)

      self.batchnorm1 = nn.BatchNorm2d(16)
      self.batchnorm2 = nn.BatchNorm2d(32)
      self.batchnorm3 = nn.BatchNorm2d(32)
      self.batchnorm4 = nn.BatchNorm1d(16)

    def forward(self, x):
      x = self.conv1(x)
      x = self.batchnorm1(x)
      x = F.relu(x)
      x = self.drop1(x)

      x = self.conv2(x)
      x = self.batchnorm2(x)
      x = F.relu(x)
      x = self.pool1(x)
      x = self.drop2(x)

      x = self.conv3(x)
      x = self.batchnorm3(x)
      x = F.relu(x)
      x = self.drop3(x)

      x = x.view(-1, 288)

      x = self.linear1(x)
      x = self.batchnorm4(x)
      x = F.relu(x)

      x = self.linear2(x)
      return F.log_softmax(x, dim=1)


In [49]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = MNISTSmallConv().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 13, 13]             160
       BatchNorm2d-2           [-1, 16, 13, 13]              32
           Dropout-3           [-1, 16, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           4,640
       BatchNorm2d-5           [-1, 32, 11, 11]              64
         MaxPool2d-6             [-1, 32, 5, 5]               0
           Dropout-7             [-1, 32, 5, 5]               0
            Conv2d-8             [-1, 32, 3, 3]           9,248
       BatchNorm2d-9             [-1, 32, 3, 3]              64
          Dropout-10             [-1, 32, 3, 3]               0
           Linear-11                   [-1, 16]           4,624
      BatchNorm1d-12                   [-1, 16]              32
           Linear-13                   [-1, 10]             170
Total params: 19,034
Trainable params: 

In [50]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [51]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [52]:
model = MNISTSmallConv().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.10167211294174194 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.05it/s]



Test set: Average loss: 0.0640, Accuracy: 9829/10000 (98.29%)



loss=0.05360158905386925 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.01it/s]



Test set: Average loss: 0.0417, Accuracy: 9874/10000 (98.74%)



loss=0.055157337337732315 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.76it/s]


...


Test set: Average loss: 0.0205, Accuracy: 9929/10000 (99.29%)



loss=0.04813915118575096 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.03it/s]



Test set: Average loss: 0.0186, Accuracy: 9937/10000 (99.37%)



loss=0.010655649937689304 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.60it/s]



Test set: Average loss: 0.0186, Accuracy: 9943/10000 (99.43%)

